In [2]:
import pandas as pd
import numpy as np
import itertools
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
import nltk
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
import re
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer  
from nltk.corpus import words
from copy import deepcopy
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec
from sklearn import svm

import tensorflow as tf
from tensorflow.keras import Sequential,optimizers
from tensorflow.keras.layers import Dense,BatchNormalization,Dropout,Activation
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import tensorflow_hub as hub

/Users/eddysmac/opt/anaconda3/envs/py38/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
class TfidfW2V():
    def __init__(self, size):
        self.vector_size = size 
        
        self.w2v = None
        
        self.vectorizer = TfidfVectorizer()
        
        self.vocab = None
        self.weights = None
    
        self.train_vector = None
        self.info_matrix = None
        
        self.nn = Sequential([
            Dense(16, input_dim=size, activation="relu"),
            Dense(32, activation="relu"),
            Dense(1, activation="sigmoid")
        ])
        self.nn.compile(
            optimizer="adam",
            loss="binary_crossentropy",
            metrics=["binary_accuracy"]
        )
        
        self.prediction = None
    
    def fit(self, X_train, X_train_corpus, y_train):
        # word embedding
        print("Preparing TFIDF weights...")
        
        tfidf = self.vectorizer.fit_transform(X_train_corpus)
        self.weights = tfidf.toarray() # the TFIDF array we generate
        self.vocab = self.vectorizer.get_feature_names() # dictionary 
        
        print("TFIDF Weights is ready, have a nice day!")
        
        
        print()
        
        
        print("Start setting Word2Vec dictionary...")
        
        self.w2v = Word2Vec(X_train, vector_size=self.vector_size, window=10, min_count=10)
        
        print("WooHoo! Word2Vec dictionary is ready!")
        
        
        print()
        
        
        print("Start generating the feature vector for training, will be a long process...")
        
        print("Setting up the information matrix... This is a better idea than for loop!")
        
        self.info_matrix = np.zeros(self.vector_size*len(self.vocab)).reshape(len(self.vocab), self.vector_size)
        
        for i in range(len(self.vocab)):
            try:
                self.info_matrix[i] = self.w2v.wv[self.vocab[i]]
            except:
                continue
            
        print("Infomation matrix is ready!!!")
        
        self.train_vector = self.weights @ self.info_matrix
        
        #for i in range(len(X_train)):
        #    self.train_vector[i] /= sum(self.weights[i])
        
        print("Feature Vector is at your service, AoLiGei!!!")
        
        print()
        
        print("Now let's welcome our great NN!!!")
        history = self.nn.fit(
            x=self.train_vector,
            y=y_train,
            batch_size=32,
            epochs=250,
            verbose=1,
            #validation_data=(X_test, y_test),
            #callbacks=[reduce_lr],
            shuffle=True
        )
    
    def predict(self, X_test, X_test_corpus, y_test):
        vectorizer = TfidfVectorizer()
        tfidf = vectorizer.fit_transform(X_test_corpus)
        weights = tfidf.toarray() # the TFIDF array we generate
        vocab = vectorizer.get_feature_names() # dictionary 
        
        print("Start generating the feature vector for training, will be a long process...")
        
        info_matrix = np.zeros(self.vector_size*len(vocab)).reshape(len(vocab), self.vector_size)
        
        for i in range(len(vocab)):
            try:
                info_matrix[i] = self.w2v.wv[vocab[i]]
            except:
                continue
        
        test_vector = weights@info_matrix
        
        #for i in range(len(X_test)):
        #    test_vector[i] /= sum(weights[i])
        
        print("Test Feature Vector is at your service, AoLiGei!!!")
        
        prediction = self.nn.predict(test_vector)
        
        for i in range(len(prediction)):
            if prediction[i] <= 0.5:
                prediction[i] = 0
            else:
                prediction[i] = 1
        
        error_count = 0
        for i in range(len(prediction)):
            if prediction[i] != y_test[i]:
                error_count += 1

        print("Error Count: {}".format(error_count))
        print("Accuracy: {}%".format(((len(y_test)-error_count)/len(y_test))*100))

In [4]:
data = pd.read_csv("movie_data.csv")
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
def preproc_text(text, stemmer="l"):
    text = re.sub(r"\d+", " ", text.lower())
    for i in string.punctuation:
        text = text.replace(i," ")
    stop_words = set(stopwords.words("english"))
    tokens = word_tokenize(text)
    words = [i for i in tokens if not i in stop_words]
    
    if not stemmer:
        # return " ".join(words)
        wordlist = words
    if stemmer == "l": # use lancasterstemmer
        stemmer = LancasterStemmer() 
        wordlist = [stemmer.stem(word) for word in words]
    elif stemmer == "w":
        stemmer = WordNetLemmatizer()
        wordlist = [stemmer.lemmatize(word) for word in words]
    
    return " ".join(wordlist) 

def spliter(text):
    return text.split(" ")

def binary_transform(sentiment):
    if sentiment == "positive":
        return 1
    return 0


In [6]:
X_corpus = data["review"].apply(preproc_text, None)
X = X_corpus.apply(spliter).values
X_corpus = X_corpus.values
y = data["sentiment"].apply(binary_transform).values

In [51]:
X_train = X[:40000]
X_train_corpus = X_corpus[:40000]
y_train = y[:40000]

X_test = X[40000:]
X_test_corpus = X_corpus[40000:]
y_test = y[40000:]

In [52]:
# check the shape of train and test
print(X_train.shape, X_train_corpus.shape, y_train.shape)
print(X_test.shape, X_test_corpus.shape, y_test.shape)

(40000,) (40000,) (40000,)
(10000,) (10000,) (10000,)


In [53]:
model = TfidfW2V(size=500)
model.fit(X_train, X_train_corpus, y_train)

Preparing TFIDF weights...
TFIDF Weights is ready, have a nice day!

Start setting Word2Vec dictionary...
WooHoo! Word2Vec dictionary is ready!

Start generating the feature vector for training, will be a long process...
Setting up the information matrix... This is a better idea than for loop!
Infomation matrix is ready!!!
Feature Vector is at your service, AoLiGei!!!

Now let's welcome our great NN!!!
Epoch 1/250
1250/1250 [==============================] - 2s 670us/step - loss: 0.3891 - binary_accuracy: 0.8272
Epoch 2/250
1250/1250 [==============================] - 1s 637us/step - loss: 0.3112 - binary_accuracy: 0.8679
Epoch 3/250
1250/1250 [==============================] - 1s 634us/step - loss: 0.3073 - binary_accuracy: 0.8710
Epoch 4/250
1250/1250 [==============================] - 1s 628us/step - loss: 0.3014 - binary_accuracy: 0.8707
Epoch 5/250
1250/1250 [==============================] - 1s 624us/step - loss: 0.3056 - binary_accuracy: 0.8682
Epoch 6/250
1250/1250 [===========

Epoch 70/250
1250/1250 [==============================] - 1s 593us/step - loss: 0.2384 - binary_accuracy: 0.8988
Epoch 71/250
1250/1250 [==============================] - 1s 590us/step - loss: 0.2362 - binary_accuracy: 0.9014
Epoch 72/250
1250/1250 [==============================] - 1s 590us/step - loss: 0.2312 - binary_accuracy: 0.9032
Epoch 73/250
1250/1250 [==============================] - 1s 605us/step - loss: 0.2424 - binary_accuracy: 0.9009
Epoch 74/250
1250/1250 [==============================] - 1s 617us/step - loss: 0.2329 - binary_accuracy: 0.9037
Epoch 75/250
1250/1250 [==============================] - 1s 657us/step - loss: 0.2354 - binary_accuracy: 0.8991
Epoch 76/250
1250/1250 [==============================] - 1s 695us/step - loss: 0.2364 - binary_accuracy: 0.9017
Epoch 77/250
1250/1250 [==============================] - 1s 600us/step - loss: 0.2348 - binary_accuracy: 0.9033
Epoch 78/250
1250/1250 [==============================] - 1s 616us/step - loss: 0.2372 - binary_

Epoch 214/250
1250/1250 [==============================] - 1s 697us/step - loss: 0.2011 - binary_accuracy: 0.9179
Epoch 215/250
1250/1250 [==============================] - 1s 721us/step - loss: 0.2027 - binary_accuracy: 0.9169
Epoch 216/250
1250/1250 [==============================] - 1s 667us/step - loss: 0.2021 - binary_accuracy: 0.9180
Epoch 217/250
1250/1250 [==============================] - 1s 691us/step - loss: 0.2050 - binary_accuracy: 0.9150
Epoch 218/250
1250/1250 [==============================] - 1s 658us/step - loss: 0.2016 - binary_accuracy: 0.9176
Epoch 219/250
1250/1250 [==============================] - 1s 672us/step - loss: 0.2058 - binary_accuracy: 0.9148
Epoch 220/250
1250/1250 [==============================] - 1s 662us/step - loss: 0.2044 - binary_accuracy: 0.9170
Epoch 221/250
1250/1250 [==============================] - 1s 650us/step - loss: 0.2047 - binary_accuracy: 0.9149
Epoch 222/250
1250/1250 [==============================] - 1s 757us/step - loss: 0.2052 

In [54]:
model.predict(X_test, X_test_corpus, y_test)

Start generating the feature vector for training, will be a long process...
Test Feature Vector is at your service, AoLiGei!!!
Error Count: 1501
Accuracy: 84.99%


In [ ]:
embed = hub.load("https://tfhub.dev/google/Wiki-words-500-with-normalization/2")